This notebook:

- is intended to be run from IBM Analytics for Apache Spark on Bluemix
- attempts to access data from a BigInsights on Cloud cluster using Hive and BigSQL.
- does not work!!! currently the bluemix spark environment does not support unlimited strength cryptography

Enter your credentials and run the notebook.  This exception probably means the spark service does not supported unlimited strength cryptography:

`javax.net.ssl.SSLKeyException: RSA premaster secret error`

# Credentials - keep this secret!

In [ ]:
cluster  = '<<changeme>>'  #  E.g. 10000
username = '<<changeme>>'  #  E.g. biadmin
password = '<<changeme>>'

# Code to connect to BigInsights on Cloud via Hive and BigSQL ...

In [ ]:
import os
cwd = os.getcwd()

cls_host = 'ehaasp-{0}-mastermanager.bi.services.bluemix.net'.format(cluster)
sql_host = 'ehaasp-{0}-master-2.bi.services.bluemix.net'.format(cluster)

Get the cluster certificate

In [ ]:
!openssl s_client -showcerts -connect {cls_host}:9443 < /dev/null | openssl x509 -outform PEM > certificate
    
# uncomment this for debugging
#!cat certificate 

Add the cluster certificate to a truststore

In [ ]:
!rm truststore.jks
!keytool -import -trustcacerts -alias biginsights -file certificate -keystore truststore.jks -storepass mypassword -noprompt

Now attempt to connect to BigInsights on Cloud

In [ ]:
# test hive
url = 'jdbc:hive2://{0}:10000/default;user={1};password={2};ssl=true;sslTrustStore={3}/truststore.jks;trustStorePassword=mypassword'.format(sql_host, username, password, cwd)
df = sqlContext.read.format('jdbc').options(url=url, driver='org.apache.hive.jdbc.HiveDriver', dbtable='GOSALESDW.SLS_SALES_FACT').load()

In [ ]:
# test bigsql
url  = 'jdbc:db2://{0}:51000/bigsql:user={1};password={2};sslConnection=true;sslTrustStoreLocation={3}/truststore.jks;Password=mypassword;'.format(sql_host, username, password, cwd)
df = sqlContext.read.format('jdbc').options(url=url, driver='com.ibm.db2.jcc.DB2Driver', dbtable='GOSALESDW.SLS_SALES_FACT').load()